In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['OMP_NUM_THREADS'] = '3' 
os.environ['KMP_BLOCKTIME'] = '1'
import numpy as np
import gc
from tqdm.auto import tqdm
from dataset import get_labelled, get_unlabelled, _preload, _unload
from Commit import CommitFactory
from Model import CommitDiffModelFactory

In [2]:
##### META PARAMS #####

#Size / depth / length of each individual context/path
CONTEXT_SIZE = 8

#Total number of contexts to sample per bag
BAG_SIZE = 256

#Size of fixed length vector to encode to
OUTPUT_SIZE = 256

# Hard limit to filter bags by. Bags which exceed this size won't even be sampled.
# Set to None to turn off
MAX_BAG_SIZE_FILTER = 2048 * 4

#Maximum number of labelled commits to train on
MAX_LABELLED_COMMITS = 4096 * 8

#Maximum number of unlabelled commits to train on
MAX_UNLABELLED_COMMITS = 4096 * 8

#Batch size for unsupervised training. Higher = faster/better but more memory required
SIAM_BATCH_SIZE = 128

In [3]:
Commit = CommitFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)
CommitDiffModel = CommitDiffModelFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, OUTPUT_SIZE=OUTPUT_SIZE)

In [4]:
_preload(max_commit_bag_size = MAX_BAG_SIZE_FILTER, max_commits = MAX_LABELLED_COMMITS)

Loading Commit lookup table


Loading commit lookups:   0%|          | 0/32768 [00:00<?, ?it/s]

Loading file ../data/commit_lookups/labelled/01_priority_commit_lookups.pickle
Appending pickle of length: 7549 , new dict length: 6906
Loading file ../data/commit_lookups/labelled/02_priority_commit_lookups.pickle
Appending pickle of length: 10410 , new dict length: 9517
Loading file ../data/commit_lookups/labelled/03_priority_commit_lookups.pickle
Appending pickle of length: 7246 , new dict length: 12646


In [5]:
X_train, X_test, y_train, y_test = get_labelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, balance=True)

Generating apache_positive_pairs X_train and y_train:   0%|          | 0/2264 [00:00<?, ?it/s]

Generating apache_negative_pairs X_train and y_train:   0%|          | 0/9511 [00:00<?, ?it/s]

In [6]:
X_train_unsupervised = get_unlabelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Unsupervised X_train:   0%|          | 0/12646 [00:00<?, ?it/s]

In [7]:
print("Train set size", len(y_train))
print("Train set split", np.sum(y_train)/len(y_train))
print("Test set size", len(y_test))
print("Test set split", np.sum(y_test)/len(y_test))
print("Unsupervised Train Size", len(X_train_unsupervised))

Train set size 3622
Train set split 0.507454445057979
Test set size 906
Test set split 0.47019867549668876
Unsupervised Train Size 12646


In [8]:
_unload()

In [ ]:
for encoder in tqdm([5,4,2,3,0,1]):
    try:
        model = CommitDiffModel(unsupervised_data_size = len(X_train_unsupervised), siam_batch_size = SIAM_BATCH_SIZE)
        model.initialize(encoder=encoder)
        model.fit_siam(np.array(X_train_unsupervised), epochs=16, verbose=1)
        model.fit_binary_classification(X_train, np.array(y_train), epochs=16, batch_size=64, verbose=1)
        score = model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)
        print("Enocder:", encoder)
        print("Score:", score)
    except Exception as e:
        print(e)
    print("------------------------------------------")
    del model
    gc.collect()

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/16
99/99 [==============================] - 36s 361ms/step - loss: -0.2507
Epoch 2/16
99/99 [==============================] - 35s 353ms/step - loss: -0.4604
Epoch 3/16
99/99 [==============================] - 36s 362ms/step - loss: -0.5127
Epoch 4/16
99/99 [==============================] - 36s 359ms/step - loss: -0.5119
Epoch 5/16
99/99 [==============================] - 36s 360ms/step - loss: -0.5064
Epoch 6/16
99/99 [==============================] - 37s 369ms/step - loss: -0.5047
Epoch 7/16
99/99 [==============================] - 36s 365ms/step - loss: -0.5059
Epoch 8/16
99/99 [==============================] - 36s 361ms/step - loss: -0.5152
Epoch 9/16
99/99 [==============================] - 36s 361ms/step - loss: -0.5230
Epoch 10/16
99/99 [==============================] - 36s 362ms/step - loss: -0.5233
Epoch 11/16
99/99 [==============================] - 36s 361ms/step - loss: -0.5223
Epoch 12/16
99/99 [==============================] - 36s 360ms/step - loss: -0.5236
E